# Simulation for theoretical MWC results

In [8]:
using Optim

"""
Mutual information (bits) as function of MWC system parameters (Martins & Swain 2011 PLoS CompBio)
* N - number of sensors
* n - number of binding sites per sensor
* K - equilibrium constant of open - close transition
* c - ratio of Kd_active / Kd_inactive
"""
function I_opt(n, N, K, c)
    return log2(sqrt((2*N)/(π*e)) * (atan(sqrt(K)) - atan(sqrt(K*(c^n)))))
end

I_opt (generic function with 1 method)

In [117]:
# Now for all settings of K and c, optimize n given N*n

K_s = logspace(0, 30, 101)
c_s = logspace(-10, 0, 101)

Ntimesn = 10000;

n_opt = zeros(length(K_s), length(c_s));
Inf_opt = zeros(length(K_s), length(c_s));
converged = zeros(length(K_s), length(c_s));

for k1 = 1:length(K_s)
    for c1 = 1:length(c_s)
        res = optimize(x->-I_opt(x, Ntimesn/x, K_s[k1], c_s[c1]), 1.0, 16)
        n_opt[k1,c1] = Optim.minimizer(res);        
        Inf_opt[k1,c1] = -Optim.minimum(res);        
        converged[k1,c1] = Optim.converged(res)
    end
end

converged;

In [122]:
using PlotlyJS

p1 = contour(x=log10(K_s), y=log10(c_s), z=round(n_opt))
p2 = contour(x=log10(K_s), y=log10(c_s), z=Inf_opt)

lo = Layout(xaxis_title="log(K)", yaxis_title="log(c)")
plot(p1, lo)

In [119]:
plot(p2, lo)

In [99]:
converged

31x31 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0 

In [195]:
# Plot the binding curve given K, c, n
function Popen(alpha, n, K, c)
    return ((1.+alpha).^n)./(K.*(1.+c.*alpha).^n .+ (1.+alpha).^n)
end

#=
# Random testing for curve plotting
inp_range = logspace(-5, 3, 1000)
outp = Popen(inp_range, 4, 10^5.44, 10^(-1.32));
inp_lin = linspace(-100, 200, 1000)*1e-3
outp_lin = Popen(0.048*exp(38*inp_lin), 4, 10^4.6, 10^(-1.32));

p1 = Plot(scatter(;x=inp_range, y=outp))
p2 = Plot(scatter(;x=log10(inp_range), y=log10(outp)))
p3 = Plot(scatter(;x=inp_lin, y=outp_lin))
p4 = Plot(scatter(;x=inp_lin, y=log10(outp_lin)))

plt = plot([p1; p2; p3; p4])
relayout!(plt, height=1000)
plt
=#

Popen (generic function with 1 method)

1000-element Array{Float64,1}:
  0.0010738 
  0.00108612
  0.00109859
  0.00111119
  0.00112395
  0.00113685
  0.00114989
  0.00116309
  0.00117644
  0.00118994
  0.0012036 
  0.00121741
  0.00123138
  ⋮         
 84.5988    
 85.5697    
 86.5518    
 87.5451    
 88.5499    
 89.5661    
 90.5941    
 91.6338    
 92.6854    
 93.7492    
 94.8251    
 95.9134    

 # Mutual info with respect to arbitrary p($\alpha$)

In [224]:
function C(a_range, p::Function, n, N, K, c; log_func=log, num_samples=10000)
    a = linspace(a_range..., num_samples)
    
    return log_func(N) .+ log_func(K) .+
    sum(p(a).*(n.*log_func(1.+a) .+ n.*log_func(1.+c.*a) 
    .- 2.*log_func(K.*(1.+c.*a).^n .+ (1.+a).^n)))
end

C (generic function with 1 method)

In [258]:
?sortperm

search: sortperm sortperm!



```rst
..  sortperm(v, [alg=<algorithm>,] [by=<transform>,] [lt=<comparison>,] [rev=false])

Return a permutation vector of indices of ``v`` that puts it in sorted order.
Specify ``alg`` to choose a particular sorting algorithm (see Sorting Algorithms).
``MergeSort`` is used by default, and since it is stable, the resulting permutation
will be the lexicographically first one that puts the input array into sorted order –
i.e. indices of equal elements appear in ascending order. If you choose a non-stable
sorting algorithm such as ``QuickSort``, a different permutation that puts the array
into order may be returned. The order is specified using the same keywords as ``sort!``.

See also :func:`sortperm!`
```


In [278]:
a_range = [0 1000]
results = zeros(100,)
for n1 = 1:100
    results[n1] = C(a_range, x1->(1/10000), n1, 10000/n1, 10^14.0, 10^-115.34)
end
@show sortperm(results, rev=true)[1]
plot(scatter(;x=1:100, y=results))

(sortperm(results,rev=true))[1] = 5


In [290]:
# Properly optimizing

K_s = logspace(0, 10, 31)
c_s = logspace(-5, 0, 31)
a_range_low = [0 1] # Corresponds to BK log10(Popen) < -3.5, ~ 80mV
a_range_med = [0 10] # Corresponds to BK log10(Popen) < -1, ~ 140mV
a_range_full = [1 100] # Corresponds to full range of BK < ~200mV

Ntimesn = 10000;

n_opt_low = zeros(length(K_s), length(c_s));
n_opt_high = zeros(length(K_s), length(c_s));
Inf_opt_low = zeros(length(K_s), length(c_s));
Inf_opt_high = zeros(length(K_s), length(c_s));
converged = zeros(length(K_s), length(c_s));

for k1 = 1:length(K_s)
    @show k1
    for c1 = 1:length(c_s)
        #= Proper optimization
        res_low = optimize(x->-C(a_range_low, x1->(1/10000), x, Ntimesn/x, K_s[k1], c_s[c1]), 1.0, 32)
        res_high = optimize(x->-C(a_range_high, x1->(1/10000), x, Ntimesn/x, K_s[k1], c_s[c1]), 1.0, 32)
        n_opt_low[k1,c1] = Optim.minimizer(res_low);
        n_opt_high[k1,c1] = Optim.minimizer(res_high)
        Inf_opt[k1,c1] = -Optim.minimum(res_high);        
        converged[k1,c1] = Optim.converged(res_high)
        =#
        
        # Compute C(n) for a set of ns, choose best
        results = zeros(50,)
        for n1 = 1:50
            results[n1] = C(a_range_low, x1->(1/10000), n1, Ntimesn/n1, K_s[k1], c_s[c1])
        end
        n_opt_low[k1,c1] = sortperm(results, rev=true)[1]
        Inf_opt_low[k1,c1] = sort(results, rev=true)[1]
        results = zeros(50,)
        for n1 = 1:50
            results[n1] = C(a_range_high, x1->(1/10000), n1, Ntimesn/n1, K_s[k1], c_s[c1])
        end
        n_opt_high[k1,c1] = sortperm(results, rev=true)[1]
        Inf_opt_high[k1,c1] = sort(results, rev=true)[1]
    end
end

converged;

k1 = 1
k1 = 2
k1 = 3
k1 = 4
k1 = 5
k1 = 6
k1 = 7
k1 = 8
k1 = 9
k1 = 10
k1 = 11
k1 = 12
k1 = 13
k1 = 14
k1 = 15
k1 = 16
k1 = 17
k1 = 18
k1 = 19
k1 = 20
k1 = 21
k1 = 22
k1 = 23
k1 = 24
k1 = 25
k1 = 26
k1 = 27
k1 = 28
k1 = 29
k1 = 30
k1 = 31


In [362]:
p1 = contour(x=log10(K_s), y=log10(c_s), z=n_opt_low)
p2 = contour(x=log10(K_s), y=log10(c_s), z=n_opt_high)

lo = Layout(xaxis_title="log(K)", yaxis_title="log(c)")
plot([Plot(p1, lo)  Plot(p2, lo)])

In [332]:
num_samples = 10000
a_range_full = [1 95]
n1 = 21; n2=4; K = 10^4.66; c= 10^-1.33
p1 = scatter(;x=linspace(a_range_low..., num_samples), y=Popen(linspace(a_range_low..., num_samples), n1, K, c))
p2 = scatter(;x=linspace(a_range_full..., num_samples), y=Popen(linspace(a_range_full..., num_samples), n2, K, c))

plot([Plot(p1); Plot(p2)])

In [354]:
plot(scatter(;x=1:1000, y=outp_lin))

In [353]:
0.048*exp(38*inp_lin)[800]

9.788061704363596

In [360]:
inp_lin[800]*1000

139.93993993993993

In [359]:
inp_lin*1000

linspace(-100.0,200.0,1000)